In [1]:
!pip install keybert
!pip install -q keybert torch pytorch-lightning transformers pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 35.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==========================================
# 맛 프로필, 키워드 설정
# [0] 라이브러리 설치 및 설정
# ==========================================
# !pip install -q keybert torch pytorch-lightning transformers pandas scikit-learn

import os
import json
import torch
import numpy as np
import glob
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import pytorch_lightning as pl
from keybert import KeyBERT # 핵심 라이브러리!

# ==========================================
# [1] 경로 설정 (⚠️ 본인 경로에 맞게 수정 필수)
# ==========================================
# 학습된 모델 파일 (.ckpt) 경로
CHECKPOINT_PATH = '/content/drive/MyDrive/Tjoproject_model/models/checkpoints/distilbert-epoch=07-val_loss=0.0219.ckpt'

# 원본 JSON 파일들이 있는 폴더
INPUT_FOLDER = '/content/drive/MyDrive/Tjoproject_model/whiskyData'

# 결과물을 저장할 새로운 폴더
OUTPUT_FOLDER = '/content/drive/MyDrive/Tjoproject_model/processed_data'

# ==========================================
# [2] 모델 클래스 정의 (불러오기 위해 필수)
# ==========================================
class WhiskyFlavorPredictor(pl.LightningModule):
    def __init__(self, n_classes=6, lr=2e-5):
        super().__init__()
        self.save_hyperparameters()
        self.bert = AutoModel.from_pretrained('distilbert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = output.last_hidden_state[:, 0]
        return self.classifier(pooled_output)

# ==========================================
# [3] 분석 함수 정의 (최종 완성본: ws_id 추가 + 구조 정리 + GPU 가속)
# ==========================================
def process_all_whiskies():
    # 0. 맛 표현 단어장 (AI가 이 중에서만 키워드를 선택함)
    FLAVOR_VOCAB = [
        "fruity", "fruit", "apple", "pear", "citrus", "lemon", "orange", "grape", "berry", "cherry", "plum", "peach", "apricot", "banana", "pineapple", "mango", "tropical", "dried fruit", "raisin", "fig", "date", "sultana", "prune",
        "sweet", "honey", "vanilla", "caramel", "toffee", "butterscotch", "chocolate", "cocoa", "sugar", "syrup", "maple", "candy", "cream", "custard", "butter", "cake", "biscuit",
        "spicy", "spice", "pepper", "cinnamon", "ginger", "clove", "nutmeg", "anise", "licorice", "mint", "herbal",
        "peaty", "peat", "smoke", "smoky", "ash", "bonfire", "charcoal", "tar", "iodine", "medicinal", "seaweed", "salt", "brine", "maritime",
        "woody", "oak", "wood", "pine", "cedar", "sawdust", "tannin", "dry", "nutty", "nut", "almond", "walnut", "hazelnut", "pecan",
        "malty", "malt", "grain", "cereal", "barley", "bread", "toast", "yeast", "dough",
        "floral", "flower", "rose", "heather", "grass", "hay", "straw", "leaf", "tea", "tobacco", "leather", "wax", "oil", "earthy", "mushroom",
        "sherry", "bourbon", "port", "rum", "wine", "cask", "finish", "rich", "smooth", "complex", "balanced"
    ]

    # 폴더 생성
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    # 1. 모델 로딩
    print("⏳ 모델들을 로딩 중입니다...")
    flavor_model = WhiskyFlavorPredictor.load_from_checkpoint(CHECKPOINT_PATH)
    flavor_model.eval()
    flavor_model.freeze()

    # GPU 장치 확인 (중요)
    device = flavor_model.device

    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    kw_model = KeyBERT()
    print(f"✅ 모델 로딩 완료! (사용 장치: {device})")

    # 2. 파일 처리 시작
    json_files = glob.glob(os.path.join(INPUT_FOLDER, "*.json"))
    print(f"\n📂 총 {len(json_files)}개의 파일을 처리합니다.\n")

    for file_path in json_files:
        try:
            file_name = os.path.basename(file_path)
            save_path = os.path.join(OUTPUT_FOLDER, file_name)

            # 중복 건너뛰기
            if os.path.exists(save_path):
                print(f"⏩ {file_name}: 이미 완료된 파일입니다. (Pass)")
                continue

            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            reviews = data.get('reviews', [])
            if not reviews:
                print(f"⚠️ {file_name}: 리뷰 없음 (Skip)")
                continue

            print(f"🥃 분석 중: {file_name} (리뷰 {len(reviews)}개)...", end="")

            # --- [Step 1] 맛 점수 프로필 계산 ---
            all_scores = []
            full_text_corpus = []

            for r in reviews:
                parts = []
                if isinstance(r, dict):
                    if r.get('nose'): parts.append(str(r.get('nose')))
                    if r.get('taste'): parts.append(str(r.get('taste')))
                    if r.get('finish'): parts.append(str(r.get('finish')))
                    if r.get('content'): parts.append(str(r.get('content')))
                    text = " ".join(parts)
                else:
                    text = str(r)

                if len(text) < 10: continue
                full_text_corpus.append(text)

                inputs = tokenizer.encode_plus(
                    text, add_special_tokens=True, max_length=256,
                    padding="max_length", truncation=True, return_tensors='pt'
                )

                # 🔥 데이터를 GPU로 이동 (에러 방지)
                input_ids = inputs['input_ids'].to(device)
                attention_mask = inputs['attention_mask'].to(device)

                with torch.no_grad():
                    pred = flavor_model(input_ids, attention_mask)
                    score = pred[0].cpu().numpy() * 10.0 # 결과는 다시 CPU로
                    all_scores.append(score)

            # 평균 점수
            if all_scores:
                avg_scores = np.mean(all_scores, axis=0)
                profile_dict = {
                    "fruity": round(float(avg_scores[0]), 1),
                    "sweet": round(float(avg_scores[1]), 1),
                    "peaty": round(float(avg_scores[2]), 1),
                    "spicy": round(float(avg_scores[3]), 1),
                    "woody": round(float(avg_scores[4]), 1),
                    "malty": round(float(avg_scores[5]), 1)
                }
            else:
                profile_dict = {}

            # --- [Step 2] 맛 키워드 추출 (단어장 기반) ---
            combined_text = " ".join(full_text_corpus)
            keywords = kw_model.extract_keywords(
                combined_text,
                candidates=FLAVOR_VOCAB, # 🌟 맛 단어장 적용
                stop_words='english',
                use_mmr=True,
                diversity=0.7,
                top_n=5
            )
            top_keywords = [kw[0] for kw in keywords]

            # --- [Step 3] 최종 데이터 구조 정리 ---

            # 1. 파일명에서 숫자 ID 추출 (예: wb_116460.json -> 116460)
            # 숫자만 남기는 안전한 방식 사용
            ws_id = "".join(filter(str.isdigit, file_name))

            # 2. info 가져오기 (없으면 빈 딕셔너리)
            final_info = data.get('info', {})

            # 3. 새로운 정보들(ID, 개수, 맛, 키워드)을 info 안에 통합
            final_info['ws_id'] = ws_id

            # collected_count가 바깥에 있으면 안으로 가져옴
            if 'collected_count' in data:
                final_info['collected_count'] = data['collected_count']
            elif 'collected_count' not in final_info:
                final_info['collected_count'] = 0

            final_info['flavor_profile'] = profile_dict
            final_info['top_keywords'] = top_keywords

            # 4. 최종 저장 (info만 저장)
            final_data = {"info": final_info} # info 키로 한 번 감싸줍니다 (요청하신 형식 유지)

            with open(save_path, 'w', encoding='utf-8') as f:
                json.dump(final_data, f, ensure_ascii=False, indent=4)

            print(f" -> ✅ 완료!")

        except Exception as e:
            print(f"\n❌ 에러 발생 ({file_path}): {e}")

    print(f"\n🎉 모든 처리가 완료되었습니다!")
    print(f"결과 파일 위치: {OUTPUT_FOLDER}")
# ==========================================
# [실행]
# ==========================================
if __name__ == '__main__':
    process_all_whiskies()

⏳ 모델들을 로딩 중입니다...
✅ 모델 로딩 완료! (사용 장치: cuda:0)

📂 총 49개의 파일을 처리합니다.

🥃 분석 중: wb_219297.json (리뷰 15개)... -> ✅ 완료!
🥃 분석 중: wb_182815.json (리뷰 57개)... -> ✅ 완료!
🥃 분석 중: wb_161049.json (리뷰 31개)... -> ✅ 완료!
🥃 분석 중: wb_165510.json (리뷰 53개)... -> ✅ 완료!
🥃 분석 중: wb_165512.json (리뷰 50개)... -> ✅ 완료!
🥃 분석 중: wb_159713.json (리뷰 51개)... -> ✅ 완료!
🥃 분석 중: wb_159714.json (리뷰 53개)... -> ✅ 완료!
🥃 분석 중: wb_211.json (리뷰 53개)... -> ✅ 완료!
🥃 분석 중: wb_2351.json (리뷰 31개)... -> ✅ 완료!
🥃 분석 중: wb_193720.json (리뷰 41개)... -> ✅ 완료!
🥃 분석 중: wb_208069.json (리뷰 50개)... -> ✅ 완료!
🥃 분석 중: wb_248393.json (리뷰 23개)... -> ✅ 완료!
🥃 분석 중: wb_32091.json (리뷰 51개)... -> ✅ 완료!
🥃 분석 중: wb_142592.json (리뷰 53개)... -> ✅ 완료!
🥃 분석 중: wb_145758.json (리뷰 33개)... -> ✅ 완료!
🥃 분석 중: wb_113049.json (리뷰 22개)... -> ✅ 완료!
🥃 분석 중: wb_158760.json (리뷰 34개)... -> ✅ 완료!
🥃 분석 중: wb_5943.json (리뷰 52개)... -> ✅ 완료!
🥃 분석 중: wb_82142.json (리뷰 10개)... -> ✅ 완료!
🥃 분석 중: wb_76497.json (리뷰 57개)... -> ✅ 완료!
🥃 분석 중: wb_71595.json (리뷰 37개)... -> ✅ 완료!
🥃 분석 중: wb_239.json

In [11]:
# ======================================================
# gemini api를 이용한 증류소명, 제품명 한글번역
# [0] 라이브러리 및 설정
# ======================================================
# !pip install -q google-generativeai

import os
import glob
import json
import time
import re
from tqdm import tqdm
import google.generativeai as genai
from google.colab import drive

# 1. 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. API 키 설정 (⚠️ 꼭 넣어야 합니다!)
MY_API_KEY = ""
genai.configure(api_key=MY_API_KEY)

MODEL_NAME = 'gemini-2.5-flash-lite'
model = genai.GenerativeModel(MODEL_NAME)
TARGET_FOLDER = '/content/drive/MyDrive/Tjoproject_model/processed_data'

# ======================================================
# [1] 헬퍼 함수: 한글 포함 여부 확인
# ======================================================
def has_korean(text):
    """문자열에 한글이 하나라도 포함되어 있는지 확인"""
    if not text: return False
    # 한글 유니코드 범위 체크
    for char in text:
        if ord('가') <= ord(char) <= ord('힣'):
            return True
    return False

# ======================================================
# [2] 번역 함수
# ======================================================
def translate_batch(batch_data):
    if not batch_data: return {}

    prompt_text = "Translate the following Whisky Names and Distilleries into Korean.\n"
    prompt_text += "Return ONLY a JSON list format: [{\"id\": \"...\", \"name_kr\": \"...\", \"dist_kr\": \"...\"}, ...]\n"
    prompt_text += "Rules: 1. 'Year Old' -> '년', 2. Phonetic translation (Glenfiddich -> 글렌피딕)\n\n"

    prompt_text += "[Input Data]\n"
    for item in batch_data:
        prompt_text += f"- ID: {item['id']} | Name: {item['name']} | Distillery: {item['dist']}\n"

    try:
        response = model.generate_content(prompt_text)
        clean_text = response.text.replace("```json", "").replace("```", "").strip()
        result_list = json.loads(clean_text)
        result_map = {item['id']: item for item in result_list if 'id' in item}
        return result_map
    except Exception as e:
        print(f"  ⚠️ 배치 API 호출 실패 (잠시 대기 후 넘어감): {e}")
        return {}

# ======================================================
# [3] 메인 실행 (수정된 로직)
# ======================================================
def fix_translations():
    json_files = glob.glob(os.path.join(TARGET_FOLDER, "*.json"))
    print(f"📂 파일 {len(json_files)}개 확인됨.")

    batch_queue = []

    print("검사 중...", end="")
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            info = data.get('info', {})
            ws_id = info.get('ws_id')

            curr_name_kr = info.get('ws_name_kr', '')

            # [조건] 1. 번역 필드가 아예 없거나  OR  2. 있는데 한글이 없을 때 -> 재작업 대상
            if not curr_name_kr or not has_korean(curr_name_kr):
                item = {
                    'id': ws_id,
                    'name': info.get('ws_name', info.get('name', 'Unknown')),
                    'dist': info.get('ws_distillery', info.get('distillery', 'Unknown')),
                    'file_path': file_path
                }
                batch_queue.append(item)

        except:
            continue

    print(f" 완료!\n🚀 재번역 대상: {len(batch_queue)}개 / {len(json_files)}개")

    if len(batch_queue) == 0:
        print("🎉 모든 파일이 이미 정상적으로 한글 번역되어 있습니다!")
        return

    # 배치 처리
    BATCH_SIZE = 20

    for i in tqdm(range(0, len(batch_queue), BATCH_SIZE), desc="복구 진행 중"):
        current_batch = batch_queue[i : i + BATCH_SIZE]

        translated_map = translate_batch(current_batch)

        # 결과 저장
        for item in current_batch:
            target_id = item['id']
            f_path = item['file_path']

            # API 결과가 없으면(실패하면) 저장하지 않음 (다음에 다시 시도하게 둠)
            if target_id not in translated_map:
                continue

            trans_res = translated_map[target_id]
            name_kr = trans_res.get('name_kr')
            dist_kr = trans_res.get('dist_kr')

            # 결과가 유효한지(한글인지) 이중 체크
            if not name_kr or not has_korean(name_kr):
                continue

            try:
                with open(f_path, 'r', encoding='utf-8') as f:
                    file_data = json.load(f)

                # 값 업데이트
                file_data['info']['ws_name_kr'] = name_kr
                file_data['info']['ws_distillery_kr'] = dist_kr

                with open(f_path, 'w', encoding='utf-8') as f:
                    json.dump(file_data, f, ensure_ascii=False, indent=4)

            except Exception as e:
                print(f"❌ 저장 실패: {e}")

        time.sleep(2)

    print("\n🎉 복구 완료! 다시 파일을 확인해보세요.")

if __name__ == "__main__":
    fix_translations()

📂 파일 49개 확인됨.
검사 중... 완료!
🚀 재번역 대상: 0개 / 49개
🎉 모든 파일이 이미 정상적으로 한글 번역되어 있습니다!
